# L1: Information retrieval

In this lab you will apply basic techniques from information retrieval to implement the core of a minimalistic search engine. The data for this lab consists of a collection of app descriptions scraped from the [Google Play Store](https://play.google.com/store/apps?hl=en). From this collection, your search engine should retrieve those apps whose descriptions best match a given query under the vector space model.

**Before starting with this lab, make sure that you have read the [Rules for hand-in assignments](https://www.ida.liu.se/~TDDE16/exam.en.shtml#handins) and the [Policy on cheating and plagiarism](https://www.ida.liu.se/~TDDE16/exam.en.shtml#cheating).** 

## Data set

The app descriptions come in the form of a compressed [JSON](https://en.wikipedia.org/wiki/JSON) file. Start by loading this file into a [Pandas](https://pandas.pydata.org) [DataFrame](https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html#dataframe).

In [1]:
import bz2
import pandas as pd

with bz2.open('app-descriptions.json.bz2') as source:
    df = pd.read_json(source)

In Pandas, a DataFrame is a table with indexed rows and labelled columns of potentially different types. Data in a DataFrame can be accessed in various ways, including by row and by column. To give an example, the code in the next cell shows rows 200–204:

In [2]:
print(len(df))
df[200:205]

1614


,name,description
200,Brick Breaker Star: Space King,Introducing the best Brick Breaker game that e...
201,Brick Classic - Brick Game,Classic Brick Game!\n\nBrick Classic is a popu...
202,Bricks Breaker - Glow Balls,Bricks Breaker - Glow Balls is a addictive and...
203,Bricks Breaker Quest,How to play\n- The ball flies to wherever you ...
204,Brothers in Arms® 3,Fight brave soldiers from around the globe on ...


As you can see, there are two labelled columns: `name` (the name of the app) and `description` (a textual description). The code in the next cell shows how to acess fields from the description column.

In [3]:
print(df.keys())
df['description'][0:5]

Index(['name', 'description'], dtype='object')


0    ﻿10000000 is a Dungeon Crawling Puzzle RPG Mat...
1    I 1177 Vårdguidens app får du tillgång till 11...
2    Need counting games for kids & drawing for tod...
3    Beautiful and simple music application for tod...
4    A Fun and intuitive numbers game for your baby...
Name: description, dtype: object

## Problem 1: Preprocessing

Your first task is to implement a preprocessor for your search engine. In the vector space model, *preprocessing* refers to any kind of transformation that is applied before a text is vectorized. Here you can restrict yourself to a very simple preprocessing: tokenization, stop word removal, and lemmatization.

To implement your preprocessor, you can use [spaCy](https://spacy.io). Make sure that you read the [Linguistic annotations](https://spacy.io/usage/spacy-101#annotations) section of the spaCy&nbsp;101; that section contains all the information that you need for this problem (and more).

Implement your preprocessor by completing the skeleton code in the next cell, adding additional code as you feel necessary.

In [4]:
import spacy

def preprocess(text):
    nlp = spacy.load("en_core_web_sm", exclude=["parser", "ner", "entity_linker", "entity_ruler"])
    
    return [token.lemma_.lower() for token in nlp(text) \
            if not(token.is_stop) and token.lemma_.isalpha()]


Your implementation should conform to the following specification:

<strong>preprocess</strong> (<em>text</em>)

> Preprocesses given text by tokenizing it, removing any stop words, replacing each remaining token with its lemma (base form), and discarding all lemmas that contain non-alphabetical characters. Returns the list of remaining lemmas (represented as strings).

**Tip:** To speed up the preprocessing, you can disable loading those spaCy components that you do not need, such as the parser, and named entity recognizer. See [here](https://spacy.io/usage/processing-pipelines#disabling) for more information about this.

Test your implementation by running the following cell:

In [5]:
preprocess('Apple is looking at buying U.K. startup for $1 billion')

['apple', 'look', 'buy', 'startup', 'billion']

This should give the following output:


## Problem 2: Vectorizing

Your next task is to vectorize the data – and more specifically, to map each app description to a tf–idf vector. For this you can use the [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) class from [scikit-learn](https://scikit-learn.org/stable/). Make sure to specify your preprocessor from the previous problem as the `tokenizer` &ndash; not the `preprocessor`! &ndash; for the vectorizer. (In scikit-learn parlance, the `preprocessor` handles string-level preprocessing.)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(tokenizer=preprocess)
X = vectorizer.fit_transform(df['description'])

Test your implementation by running the following cell:

In [7]:
X.shape

(1614, 21427)

This should show the dimensions of the matrix `X` to be 1614 × 21427.

## Problem 3: Retrieving

To complete the search engine, your last task is to write a function that returns the most relevant app descriptions for a given query. An easy way to do solve this task is to use scikit-learn&rsquo;s [NearestNeighbors](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html) class. That class implements unsupervised nearest neighbours learning, and allows you to easily find a predefined number of app descriptions whose vector representations are closest to the query vector.

In [8]:
from sklearn.neighbors import NearestNeighbors

def search(query):
    neigh = NearestNeighbors(n_neighbors=10, radius=0.4)
    neigh.fit(X)
    query_idf = vectorizer.transform([query])
    neighbours = neigh.kneighbors(query_idf, return_distance=False)
    
    return df.iloc[[i for i in neighbours[0]]]

Your implementation should conform to the following specification:

<strong>search</strong> (<em>query</em>)

> Returns the 10 app descriptions most similar (in terms of cosine similarity) to the given query as a Pandas DataFrame.

Test your implementation by running the following cell:

In [9]:
search('dodge trains')

,name,description
1301,Subway Surfers,DASH as fast as you can! \nDODGE the oncoming ...
1428,Train Conductor World,Master and manage the chaos of international r...
1394,Tiny Rails,All aboard for an adventure around the world!\...
1300,Subway Princess Runner,"Subway princess runner, Bus run, forest rush w..."
998,No Humanity - The Hardest Game,2M+ Downloads All Over The World!\n\n* IGN Nom...
1429,Train for Animals - BabyMagica free,"🚂 BabyMagica ""Train for Animals"" is a educatio..."
1168,Rush,Are you ready for a thrilling ride?\n\nRush th...
786,LEGO® DUPLO® Train,All aboard! Driving the colorful LEGO® DUPLO® ...
1465,Virus War - Space Shooting Game,Warning! Virus invasion! Destroy them with you...
192,Boxing Star,Go for the K.O.!\nMake Your Opponent See Stars...


The top hit in the list should be *Subway Surfers*.

## Problem 4: Finding terms with low/high idf

Recall that the inverse document frequency (idf) of a term is the lower the more documents from a given collection the term appears in. To get a better understanding for this concept, your next task is to write code to find out which terms have the lowest/highest idf with respect to the app descriptions.

Start by sorting the terms in increasing order of idf, breaking ties by falling back on alphabetic order.

In [26]:
terms = vectorizer.get_feature_names_out()
idf_sums = vectorizer.idf_

# Pair up terms with their idf sums. Sorted on ascending
data = [(term, idf_sums[col]) for col, term in enumerate(terms)]
term_value_pairs = sorted(data, key=lambda pair: (pair[1], pair[0]))

terms = [pair[0] for pair in term_value_pairs]

Now, print the 10 terms with the lowest/highest idf. How do you explain the results?

In [27]:
print('First 10:')
print(terms[:10])

print('\nLast 10:')
print(terms[-10:])

print('\n\nThe first 10 terms, with the lowest idf values, are the most commonly occuring' + 
     '\nin the documents')
print('\nThe last 10 terms, with the highest idf values, occurs in fewest documents' + 
      '\n(probably only in one of them). Mostly Korean(?) words because its a '+
      '\nlanguage not used much in the given context.')


First 10:
['game', 'play', 'feature', 'free', 'new', 'world', 'time', 'app', 'fun', 'use']

Last 10:
['회원가입에', '회원을', '획득한', '효과', '효과음', 'ﬁnd', 'ﬁnger', 'ﬁnish', 'ﬁrst', 'ﬂye']


The first 10 terms, with the lowest idf values, are the most commonly occuring
in the documents

The last 10 terms, with the highest idf values, occurs in fewest documents
(probably only in one of them). Mostly Korean(?) words because its a 
language not used much in the given context.


## Problem 5: Keyword extraction

A simple method for extracting salient keywords from a document is to pick the $k$ terms with the highest tf–idf value. Your last task in this lab is to implement this method. More specifically, we ask you to implement a function `keywords` that extracts keywords from a given text.

In [24]:
import numpy as np

def keywords(text, n=10):
    term_array = np.array(vectorizer.get_feature_names_out())
    vectorized_query = vectorizer.transform([text]).toarray().flatten()
    
    
    # Debug: Printing the tf-idf values
    pairs = [
        (term_array[i], vectorized_query[i]) \
         for i in range(len(vectorized_query)) \
         if vectorized_query[i] > 0.0
    ]
    print(*sorted(pairs, key=lambda x: x[1])[-1:-n-1:-1], sep='\n', end='\n'*3)
    
    
    sorted_index = np.argsort(vectorized_query)
    return term_array[sorted_index[-1:-n-1:-1]]

Your implementation should conform to the following specification:

<strong>keywords</strong> (<em>text</em>, <em>n</em> = 10)

> Returns a list with the $n$ (default value: 10) most salient keywords from the specified text, as measured by their tf–idf value relative to the collection of app descriptions.

Test your implementation by running the following cell:

In [25]:
print(keywords(df['description'][1428]))

('train', 0.3668995405502609)
('railway', 0.21071250638620947)
('railroad', 0.21071250638620947)
('rail', 0.19439227146214436)
('chaos', 0.17561322826201356)
('crash', 0.15529418865429875)
('tram', 0.1229058922552027)
('timetable', 0.1229058922552027)
('railyard', 0.1229058922552027)
('overcast', 0.1229058922552027)


['train' 'railway' 'railroad' 'rail' 'chaos' 'crash' 'timetable' 'haul'
 'overcast' 'locomotive']


This should give the following output:

## Reflection questions

The following reflection questions are questions that you could be asked in the oral exam. Try to answer each of them in the form of a short text and put it in the cell below. You will get feedback on your answers from your lab assistant.

**RQ 1.1:** Why do we remove common stop words and lemmatise the text? Can you give an example of a scenario where, in addition to common stopwords, there are also *domain-specific* or *application-specific* stop words?

> We remove stop words and lemmatise the text to make the sparse vector representation smaller and more targeted. Stop words are grammar specific and are widely used everywhere in text, thus they will inevitably be the most frequently used words (distributed according to zipfs law). However, they provide no topical meaning in the given context and will only be overshadowing the more relevant words, unnecessarily waste memory resources, and slow down search time. 
Simularly, there are many grammatical forms for a given word, but the meaning is always more or less the same. Thus, converting all possible variations of a word to its lemma representation will remove redundancy, make the term frequency based representation faster, and more accurate.

> *Domain-specific* stop words are words that will often occur in a given context, but not provide any relevant meaning to the text. The dataset for this lab is the "app description" dataset, and one of the most common word is "app". This is a typical word in this context and do not provide any useful meaning to the description and can thus be considered a *Domain-specific* stop word. Another similar example would be the word "movie" or "actor" in a dataset based on movie descriptions.


**RQ 1.2:** In Problem&nbsp;2, what do the dimensions of the matrix `X` correspond to? This matrix gives rise to different types of *representations*. Explain what these representations are. What information from the data is preserved, what information is lost in these representations?

> Every entry in the array represents a document, and every document is represented with a sparse vector containing tf_idf value for all terms from the trained vectorizer, in order. Meaning: if a term does not exist in the document, it will be represented by a zero, and if it does exist, it will be a weighted value based on the "term frequency inverse document frequency" value. It is meant to represent how important a word is to a document by combining the term frequency with the inverse document frequency which attempts to give more discriminative power to terms by penalizing generic words, and thus the information about frequency is partly lost due to this compensation.

**RQ 1.3:** What does it mean that a term has a high/low idf value? Based on this, how can we use idf to automatically identify stop words? Why do you think is idf not used as a term weighting on its own, but always in connection with term frequency (tf–idf)?

> A high idf value on a term means that it is very frequently used in the given dataset. This implies that it might not provide any unique and/or relevant meaning in the context. Usually it would indicate that the word is a stop word. It could be a standard grammatical stop word or a domain-specific stop word. As such, it is possible to automatically identify these stop words by looking at the terms with the highest idf values.
However, sometimes terms with high idf values provides relevant meaning as well. For example: The dataset for this lab had "game" as the most common term (excluding the already removed grammatical stop words), but it is actually a relevant term which says that this app is a game. Not all apps are games, so it could be useful information in the given context.

> Only using the idf value means that there is no distinguishing between a term that only occurs once in a document and a term that occurs many times in that document. If the document is a description of a train game, then the term 'train' will most likely occur many times, but this information will be lost and overshadowed with other terms with higher idf values, like for example, some commonly used adjectives. Since 'train' is something we would expect to find in that context, using the tf to enhance the idf values here is highly relevant.

*TODO: Enter your answers here*

**Congratulations on finishing L1! 👍**